## Preprocessing IAS CoRoT lightcurves for real time spatialized Sonification

Data download: http://idoc-corot.ias.u-psud.fr/sitools/client-user/COROT_N2_PUBLIC_DATA/project-index.html

The CoRoT space mission, launched on 2006 December 27, was developed and operated by the CNES, with participation of the Science Programs of ESA, ESA's RSSD, Austria, Belgium, Brazil, Germany and Spain.


In [ ]:
from astropy.io import fits
from astropy.timeseries import BoxLeastSquares
import matplotlib.pylab as plt
import numpy as np

import os
from pathlib import Path

In [ ]:
from astropy.coordinates import SkyCoord
from astropy.visualization.wcsaxes.frame import EllipticalFrame
from astropy.coordinates import Angle, SkyCoord
from mocpy import MOC, World2ScreenMPL
import astropy.units as u

In [ ]:
import session_info
session_info.show()

In [ ]:
root = "#####YOUR PATH TO THE DOWNLOADED LIGHT CURVE FOLDER#####"
file = "EN2_STAR_MON_0400007538_20111201T181527_20120109T103228.fits"
path = root + "/" + file

### Opening one light curve and looking at the data

In [ ]:
fits.getdata(path, ext=1).columns
with fits.open(path, mode="readonly") as hdulist:
    jds = hdulist[1].data['DATETT']
    white_flux = hdulist[1].data['WHITEFLUX']
    mk = hdulist[0].header['SPECTYPE']
    lum_class = hdulist[0].header['LUMCLASS']
    rms = hdulist[0].header['LC_RMS']
    mean = hdulist[0].header['LC_mean']
    ra = hdulist[0].header['ALPHA']
    dec = hdulist[0].header['DELTA'] 
    data_dim = hdulist[1].header['NAXIS2']
    
hdulist.close()  

model = BoxLeastSquares(jds, white_flux)
results = model.autopower(0.16)
best_fit = results.period[np.argmax(results.power)]      
    
fig, ax = plt.subplots()
fig.set_size_inches(12., 8.)
ax.plot(jds, white_flux, 'r.')
ax.set_ylim((mean - 10000), (mean + 10000))

fig.suptitle(file)
ax.set_ylabel("White Flux (e-/32s)")
ax.set_xlabel("Time (JD)")

print(file)
print("Spectral type: ", mk)
print("Luminosity class: ", lum_class)
print("LC RMS: ", rms)
print("LC Mean: ", mean)
print("RA: ", ra)
print("DEC: ", dec)
print("Bes fit period: ", round(best_fit, 3), "days")

### Counting the files in the archive

In [ ]:
num = 1
dim1 = data_dim
for path, subdirs, files in os.walk(root):
    for name in files:
        num += 1        
print(num)
print(dim1)

### Loading the complete CoRoT archive

In [ ]:
curves = 0
flux_data = []
jds_data = []

lc_set = [''] * num 

mk_list = []
lumclass_list = []
rms_list = []
mean_list = []
ra_list = []
dec_list = []
period_list = []
names_list = []
dimensions_list = []

for path, subdirs, files in os.walk(root):
    for name in files:
        file = [os.path.join(path, name)]
        str = " " 
        Ffile = (str.join(file))
        route = Path(name)
        Fname = route.with_suffix('')
        Fpng = route.with_suffix('.png')
        
        print("____________________________________")
        print("Loading data: light curve ", curves+1)
        print(name)
        names_list.append(name)
        fits.getdata(Ffile, ext=1).columns
        with fits.open(Ffile, mode="readonly") as hdulist:
 
            try:
                jds = hdulist[1].data['DATETT']
                white_flux = hdulist[1].data['WHITEFLUX']
                mk = hdulist[0].header['SPECTYPE']
                lum_class = hdulist[0].header['LUMCLASS']
                rms = hdulist[0].header['LC_RMS']
                mean = hdulist[0].header['LC_mean']
                ra = hdulist[0].header['ALPHA']
                dec = hdulist[0].header['DELTA'] 
                dimension = hdulist[1].header['NAXIS2']
            except:
                print("-------------------------------")
                print("Error loading mean and rms. Recalculating values")
                jds = hdulist[1].data['DATETT']
                white_flux = hdulist[1].data['WHITEFLUX']
                mk = hdulist[0].header['SPECTYPE']
                lum_class = hdulist[0].header['LUMCLASS']
                rms = np.sqrt(np.mean(white_flux**2))
                mean = np.mean(white_flux)
                ra = hdulist[0].header['ALPHA']
                dec = hdulist[0].header['DELTA'] 
                dimension = hdulist[1].header['NAXIS2']


        hdulist.close()
        
        #Box Least Squares Periodogram analysis
        model = BoxLeastSquares(jds, white_flux)
        results = model.autopower(0.2) #0.16
        best_fit = results.period[np.argmax(results.power)] 

        
        flux_data.append(white_flux)
        jds_data.append(jds)
        mk_list.append(mk)
        lumclass_list.append(lum_class)
        rms_list.append(rms)
        mean_list.append(mean)
        ra_list.append(ra)
        dec_list.append(dec)
        period_list.append(best_fit)
        dimensions_list.append(dimension)
          
        median = np.median(white_flux)
        
        fig, ax = plt.subplots()
        fig.set_size_inches(12., 8.)
        ax.plot(jds, white_flux, 'r.')
        ax.set_ylim((median - 10000), (median + 10000))

        fig.suptitle(name)
        ax.set_ylabel("White Flux (e-/32s)")
        ax.set_xlabel("Time (JD)")
        plt.show()
        
        print("--------Lightcurve ", curves+1, "--------")
        print("Spectral type: ", mk)
        print("Luminosity class: ", lum_class)
        print("LC RMS: ", rms)
        print("LC Mean: ", mean)
        print("RA: ", ra)
        print("DEC: ", dec)
        print("Bes fit period: ", round(best_fit, 3), "days")
        print(f'Array dimension: {dimension} points')

        lc_set[curves] = name
        curves += 1
               
    print ("Light curves loaded: ",curves+1);


### Plotting the Footprint

In [ ]:
coords = SkyCoord(ra_list,dec_list,frame='icrs',unit='deg')

In [ ]:
fig = plt.figure(figsize=(16,8))

with World2ScreenMPL(
    fig,
    fov=320 * u.deg,
    center=SkyCoord(100, 0, unit='deg', frame='icrs'),
    coordsys="icrs",
    rotation=Angle(0, u.degree),
) as wcs:
    ax = fig.add_subplot(111, projection=wcs, frame_class=EllipticalFrame)
    ax.set_title("CoRoT library")
    ax.grid(color="black", linestyle="dotted")
    ax.scatter(coords.ra,coords.dec,marker='o', s=10,transform=ax.get_transform('world'), zorder=10)
    ax.scatter(100 ,9 ,marker='*',color = 'r', s=100,transform=ax.get_transform('world'),zorder=10)

plt.savefig('CoRot_footprint.png', transparent=True)

    

### Data overview

In [ ]:
mk_list

In [ ]:
mk_list[0][0]# note mapping O=C B=B A=A F=F G=G K=D M=E

In [ ]:
mk_list[0][1]# octave

In [ ]:
lumclass_list

In [ ]:
lumclass_list[4] == "IV"

In [ ]:
mean_list

In [ ]:
print("Mean Max: ", max(mean_list)) # normalized (0.2 - 1)
print("Mean Min: ",min(mean_list))

In [ ]:
rms_list

In [ ]:
period_list

In [ ]:
print("Mean Max: ", max(period_list)) # invert => frequency
print("Mean Min: ",min(period_list))

### Saving to provide real time sonification

In [ ]:
np.save('flux_data', flux_data)

In [ ]:
np.save('jds_data', jds_data)

In [ ]:
np.save('mk_list.npy', mk_list)

In [ ]:
np.save('lumclass_list', lumclass_list)

In [ ]:
np.save('mean_list', mean_list)

In [ ]:
np.save('rms_list', rms_list)

In [ ]:
np.save('period_list', period_list)

In [ ]:
np.save('ra_list', ra_list)

In [ ]:
np.save('dec_list', dec_list)

In [ ]:
np.save('dimensions_list', dimensions_list)

In [ ]:
np.save('names_list', names_list)